# 1. Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2. Data loading

## 2.1. Dataset description

### A toy dataset
- **Triangular** currency pairs: `gbpusd`, `usdjpy`, **`jpygbp`**
  - $\frac{USD}{GBP}\times\frac{JPY}{USD}\times\frac{GBP}{JPY}=1$. Therefore, the following holds assuming a frictionless market condition:
  - $\log{\frac{USD}{GBP}}-\log{\frac{JPY}{USD}}-\log{\frac{GBP}{JPY}}=0.$
- Period: One day. May 1st, 2019.
- Frequency: 1 minute (=1440 data points per day per currency pair)

## 2.2 Loading a toy data set

In [125]:
toy_path = '../dataset/'
toy_folders = ['toyset']

# csv files have no explicit column header, but column names come in this order.
# 'timestamp',  'opening', 'high', 'low', 'close', 'volume'
# As we are only interested in timestamp and close prices, we set `usecols` = [0,4], a list of indices
col_names = ['timestamp',  'opening', 'high', 'low', 'close', 'volume']
usecols = [0, 4]

df = {}
f = pd.DataFrame(columns=col_names)
print("Loading...")
for folder in toy_folders:
    files = os.listdir(toy_path+folder)
    for file in files:
        if file.endswith(".csv"):
            print(file)
            tmp = pd.read_csv(os.path.join(toy_path, folder, file),
                              delimiter=';', header=0, names=col_names, usecols=usecols)
            df[file[:6]] = tmp.copy()
print("Complted.")

Loading...
gbpjpy_DAT_ASCII_GBPJPY_M1_201905.csv
gbpusd_DAT_ASCII_GBPUSD_M1_201905.csv
usdjpy_DAT_ASCII_USDJPY_M1_201905.csv
Complted.


In [126]:
df.keys()

dict_keys(['gbpjpy', 'gbpusd', 'usdjpy'])

#### Conversion: gbpjpy -> jpygbp
- In the raw data set, only gbpjpy exists but we need `jpygbp`
  - Very easy to convert: the reciprocal of gbpjpy is `jpygbp`

In [127]:
df['jpygbp'] = df['gbpjpy'].copy()
df['jpygbp'].close = df['jpygbp'].close.apply(lambda x: 1.0/x)
df.pop('gbpjpy', None)

,timestamp,close
0,20190501 000100,145.451
1,20190501 000200,145.465
2,20190501 000300,145.459
3,20190501 000400,145.453
4,20190501 000500,145.450
...,...,...
31945,20190531 165400,136.769
31946,20190531 165500,136.765
31947,20190531 165600,136.772
31948,20190531 165700,136.776


In [128]:
cry_list = list(df.keys())
print('We have a list of currency pairs:', cry_list)

We have a list of currency pairs: ['gbpusd', 'usdjpy', 'jpygbp']


## 2.3 Cleaning data

### Drop all data points other data May 1, 2019
- Also take logs and create a `log_close` column to save log values.
- Now we have final observed data points in this column `log_close`
- Create a list of numpy(`log_close`) for three currency pairs for easier usage and manipulation

In [129]:
begin_date = '20190501 000100'
end_date = '20190501 235900'

#### We asuume a batch size of 1,440 (minutes)

In [130]:
batch_size = 60*24

In [131]:
idx = pd.DataFrame(pd.Series(pd.date_range(
    begin_date[:8], periods=batch_size, freq='1min')), columns=['datetime'])

In [132]:
for cry in cry_list:
    df[cry] = df[cry][df[cry].timestamp.str.startswith('20190501')]
    df[cry]['datetime'] = df[cry].timestamp.apply(lambda x: pd.to_datetime(x))
    df[cry][cry+'log_close'] = df[cry].close.apply(lambda x: np.log(x))
    idx = idx.merge(df[cry], how='left', left_on='datetime', right_on='datetime')

In [133]:
idx

,datetime,timestamp_x,close_x,gbpusdlog_close,timestamp_y,close_y,usdjpylog_close,timestamp,close,jpygbplog_close
0,2019-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-05-01 00:01:00,20190501 000100,1.30428,0.265651,20190501 000100,111.488,4.713917,20190501 000100,0.006875,-4.979839
2,2019-05-01 00:02:00,20190501 000200,1.30439,0.265735,20190501 000200,111.489,4.713926,20190501 000200,0.006875,-4.979936
3,2019-05-01 00:03:00,20190501 000300,1.30437,0.265720,20190501 000300,111.489,4.713926,20190501 000300,0.006875,-4.979894
4,2019-05-01 00:04:00,20190501 000400,1.30428,0.265651,20190501 000400,111.489,4.713926,20190501 000400,0.006875,-4.979853
...,...,...,...,...,...,...,...,...,...,...
1435,2019-05-01 23:55:00,20190501 235500,1.30544,0.266540,20190501 235500,111.540,4.714383,20190501 235500,0.006867,-4.980966
1436,2019-05-01 23:56:00,20190501 235600,1.30550,0.266586,20190501 235600,111.542,4.714401,20190501 235600,0.006867,-4.981021
1437,2019-05-01 23:57:00,20190501 235700,1.30546,0.266555,20190501 235700,111.547,4.714446,20190501 235700,0.006867,-4.981028
1438,2019-05-01 23:58:00,20190501 235800,1.30544,0.266540,20190501 235800,111.547,4.714446,20190501 235800,0.006867,-4.981042


# 3. Extended Kalman Filter

## 3.1 Problem setup

- $\mathbf{X}$ is the hidden state estimate and we take it as the estimate of currency intrinsic values.
- $\mathbf{Y}_t$ is the observed values of FX rates at time step $t$.
  - An observed data point, i.e., an FX rate is denoted by $y_t^{c_i,c_j}$ for a time step $t$, where $c_ic_j \in C := \{gbpusd, usdjpy, jpygbp\}$ and $i\neq j. 1=gbp, 2=usd, 3=jpy$.
  - Three data points: $y_t^{gbpusd}, y_t^{usdjpy}, y_t^{jpygbp}$ at a time step $t$
  - $\mathbf{Y}_t = [y_t^{gbpusd}, y_t^{usdjpy}, y_t^{jpygbp}]$
- We observe data for time step $1:t$, so $\mathbf{Y}$ is a (T, 3) matrix.

## 3.2 Modeling

- Now think of a simple model:
$$\mathbf{X}_{t+1} = \mathbf{f}(\mathbf{X}_t) + \mathbf{Q}_t$$
$$\mathbf{Y}_t = \mathbf{h}(\mathbf{X}_t)+\mathbf{R}_t$$,
where $\mathbf{f}$ and $\mathbf{h}$ are non-linear functions. If they were linear, it would have been

#### Approximation of $\mathbf{f(X_t)}$ using Taylor's expansion:
$$\mathbf{f(X_t)} \approx \mathbf{f}(p)+\nabla \mathbf{f}|_p(\mathbf{X}_t-p)=\mathbf{f}(p)+ \mathbf{H}_t(\mathbf{X}_t-p)$$
, where $p=x_{t}^{est}$, the previous optimal estimate, and $\mathbf{H}_t$ is the Jacobian matrix at time step $t$.

According to the Kalman Filter example in the Pyro website, now we have predictions and updates rules as follows:

#### The predictions are:

$$\hat{\mathbf{X}}_t \approx \mathbf{f}(\mathbf{X}_{t-1}^{est})$$
$$\mathbf{Y}_t = \mathbf{H_f}(\mathbf{X}_{t-1})\mathbf{Y}_{t-1}\mathbf{H_f}^T(\mathbf{X}_{t-1})+\mathbf{R}$$

#### The updates are:

$$\mathbf{X_t} \approx \hat{\mathbf{X}}_t+\mathbf{K}_t\left(y_t-\mathbf h(\hat{\mathbf{X}}_t)\right)$$
$$K_t=\hat{P}_t\mathbf{H_h}(\hat{\mathbf{X}}_t)\left(\mathbf{H_h}(\hat{\mathbf{X}}_t)\hat{P}_t\mathbf{H_h}(\hat{\mathbf{X}}_t)+R_t\right)^{-1}$$
$$P_t = \left(I-K_t\mathbf{H_h}(\hat{\mathbf{X}}_t)\right)\hat{P}_t$$

## Pyro implementation
- I follow the example in Pyro website to practice for now.

In [ ]:
import os
import math

import torch
import pyro
import pyro.distributions as dist
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO, config_enumerate
from pyro.contrib.tracking.extended_kalman_filter import EKFState
from pyro.contrib.tracking.distributions import EKFDistribution
from pyro.contrib.tracking.dynamic_models import NcvContinuous
from pyro.contrib.tracking.measurements import PositionMeasurement

- According to the following link, having the two lines of codes are the **best practices** without further explanation: https://github.com/pyro-ppl/pyro/issues/930

In [46]:
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('0.4.1')
pyro.enable_validation(True)

In [49]:
dt = 1e-2
num_frames = 10
dim = 4

# Continuous model
ncv = NcvContinuous(dim, 2.0)

# Truth trajectory
xs_truth = torch.zeros(num_frames, dim)
# initial direction
theta0_truth = 0.0
# initial state
with torch.no_grad():
    xs_truth[0, :] = torch.tensor(
        [0.0, 0.0,  math.cos(theta0_truth), math.sin(theta0_truth)])
    for frame_num in range(1, num_frames):
        # sample independent process noise
        dx = pyro.sample('process_noise_{}'.format(
            frame_num), ncv.process_noise_dist(dt))
        xs_truth[frame_num, :] = ncv(xs_truth[frame_num-1, :], dt=dt) + dx

In [50]:
xs_truth

tensor([[ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 9.9298e-03, -4.2019e-05,  9.9879e-01, -6.2927e-03],
        [ 1.9858e-02, -1.3703e-04,  9.8131e-01, -6.4835e-03],
        [ 2.9670e-02, -1.3280e-04,  9.8039e-01,  5.1214e-03],
        [ 3.9381e-02,  9.5771e-06,  9.5822e-01,  2.0339e-02],
        [ 4.9010e-02,  1.4353e-04,  9.6450e-01,  1.8807e-02],
        [ 5.8850e-02,  2.9863e-04,  9.8926e-01,  1.6081e-02],
        [ 6.8766e-02,  5.4268e-04,  9.9629e-01,  2.1323e-02],
        [ 7.8719e-02,  7.1544e-04,  1.0032e+00,  1.4204e-02],
        [ 8.8778e-02,  8.4070e-04,  1.0087e+00,  1.1806e-02]])

<function numpy.cov(m, y=None, rowvar=True, bias=False, ddof=None, fweights=None, aweights=None)>

In [63]:
measurements = []
mean = torch.zeros(2)
# no correlations
cov = 1e-5 * torch.eye(2)
i=1
with torch.no_grad():
    # sample independent measurement noise
    dzs = pyro.sample('dzs', dist.MultivariateNormal(mean, cov).expand((num_frames,)))
    print("#{:2d}".format(i))
    print('dsz', dzs)
    # compute measurement means
    zs = xs_truth[:, :2] + dzs
    print('zs', zs)

# 1
dsz tensor([[ 1.9301e-03,  1.4764e-03],
        [ 6.1685e-03, -3.3618e-03],
        [-2.6227e-03, -4.4501e-03],
        [ 5.1445e-03,  5.4478e-04],
        [-5.0960e-03, -1.5161e-03],
        [-4.5332e-04, -1.0034e-03],
        [ 1.8141e-03,  3.1557e-03],
        [ 1.7190e-03,  2.4920e-04],
        [ 2.7286e-03, -6.1632e-05],
        [ 3.1340e-03, -2.4594e-03]])
zs tensor([[ 0.0019,  0.0015],
        [ 0.0161, -0.0034],
        [ 0.0172, -0.0046],
        [ 0.0348,  0.0004],
        [ 0.0343, -0.0015],
        [ 0.0486, -0.0009],
        [ 0.0607,  0.0035],
        [ 0.0705,  0.0008],
        [ 0.0814,  0.0007],
        [ 0.0919, -0.0016]])


In [52]:
def model(data):
    # a HalfNormal can be used here as well
    R = pyro.sample('pv_cov', dist.HalfCauchy(2e-6)) * torch.eye(4)
    Q = pyro.sample('measurement_cov', dist.HalfCauchy(1e-6)) * torch.eye(2)
    # observe the measurements
    pyro.sample('track_{}'.format(i), EKFDistribution(xs_truth[0], R, ncv,
                                                      Q, time_steps=num_frames),
                obs=data)

guide = AutoDelta(model)  # MAP estimation

In [66]:
optim = pyro.optim.Adam({'lr': 2e-2})
svi = SVI(model, guide, optim, loss=Trace_ELBO(retain_graph=True))

pyro.set_rng_seed(0)
pyro.clear_param_store()

for i in range(250 if not smoke_test else 2):
    loss = svi.step(zs)
    if not i % 10:
        print('#{:3d} loss: {}'.format(i, loss))

#  0 loss: -16.5052490234375
# 10 loss: -16.558531761169434
# 20 loss: -16.588309288024902
# 30 loss: -16.603419303894043
# 40 loss: -16.61054801940918
# 50 loss: -16.613832473754883
# 60 loss: -16.6153564453125
# 70 loss: -16.616074562072754
# 80 loss: -16.61642074584961
# 90 loss: -16.61659049987793
#100 loss: -16.616674423217773
#110 loss: -16.61671733856201
#120 loss: -16.616735458374023
#130 loss: -16.616744995117188
#140 loss: -16.616750717163086
#150 loss: -16.61675262451172
#160 loss: -16.616753578186035
#170 loss: -16.616753578186035
#180 loss: -16.616753578186035
#190 loss: -16.616756439208984
#200 loss: -16.616753578186035
#210 loss: -16.616755485534668
#220 loss: -16.61675453186035
#230 loss: -16.61675453186035
#240 loss: -16.61675453186035


In [54]:

# retrieve states for visualization
R = guide()['pv_cov'] * torch.eye(4)
Q = guide()['measurement_cov'] * torch.eye(2)
ekf_dist = EKFDistribution(xs_truth[0], R, ncv, Q, time_steps=num_frames)
states= ekf_dist.filter_states(zs)

In [55]:
states

In [56]:
R

tensor([[1.4523e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.4523e-06, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.4523e-06, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4523e-06]],
       grad_fn=<MulBackward0>)

In [57]:
Q

tensor([[2.5117e-07, 0.0000e+00],
        [0.0000e+00, 2.5117e-07]], grad_fn=<MulBackward0>)

In [58]:
ekf_dist

EKFDistribution(measurement_cov: torch.Size([2, 2]), P0: torch.Size([4, 4]), x0: torch.Size([4]))